In [51]:
import torchvision as tv
import pandas as pd
import numpy as np
import time
import torch

In [52]:
BATCH_SIZE = 512

In [56]:
train_dataset = tv.datasets.FashionMNIST('.', train = True, transform = tv.transforms.ToTensor(), download = True)
test_dataset = tv.datasets.FashionMNIST('.', train = False, transform = tv.transforms.ToTensor(), download = True)
train = torch.utils.data.DataLoader(train_dataset, batch_size = BATCH_SIZE)
test = torch.utils.data.DataLoader(test_dataset, batch_size = BATCH_SIZE)

In [57]:
train = torch.utils.data.DataLoader(train_dataset, batch_size = BATCH_SIZE)
test = torch.utils.data.DataLoader(test_dataset, batch_size = BATCH_SIZE)

In [58]:
train_dataset[0][0].shape

torch.Size([1, 28, 28])

In [94]:
model = torch.nn.Sequential(
    torch.nn.Flatten(),
    torch.nn.Linear(784,512),
    torch.nn.ReLU(),
    torch.nn.BatchNorm1d(512),
    torch.nn.Linear(512,256),
    torch.nn.ReLU(),
    torch.nn.BatchNorm1d(256),
    torch.nn.Linear(256, 128),
    torch.nn.ReLU(),
    torch.nn.BatchNorm1d(128),
    torch.nn.Linear(128, 64),
    torch.nn.ReLU(),
    torch.nn.BatchNorm1d(64),
    torch.nn.Linear(64, 32),
    torch.nn.ReLU(),
    torch.nn.BatchNorm1d(32),
    torch.nn.Linear(32,10))

In [95]:
model

Sequential(
  (0): Flatten(start_dim=1, end_dim=-1)
  (1): Linear(in_features=784, out_features=512, bias=True)
  (2): ReLU()
  (3): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (4): Linear(in_features=512, out_features=256, bias=True)
  (5): ReLU()
  (6): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (7): Linear(in_features=256, out_features=128, bias=True)
  (8): ReLU()
  (9): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (10): Linear(in_features=128, out_features=64, bias=True)
  (11): ReLU()
  (12): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (13): Linear(in_features=64, out_features=32, bias=True)
  (14): ReLU()
  (15): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (16): Linear(in_features=32, out_features=10, bias=True)
)

In [96]:
loss = torch.nn.CrossEntropyLoss()
trainer = torch.optim.Adam(model.parameters(), lr = 0.01)
num_epoch = 100

In [97]:
def train_model():
    for ep in range(num_epoch):
        train_iters, train_passed  = 0, 0
        train_loss, train_acc = 0., 0.
        start=time.time()
        
        model.train()
        for X, y in train:
            trainer.zero_grad()
            y_pred = model(X)
            l = loss(y_pred, y)
            l.backward()
            trainer.step()
            train_loss += l.item()
            train_acc += (y_pred.argmax(dim=1) == y).sum().item()
            train_iters += 1
            train_passed += len(X)
        
        test_iters, test_passed  = 0, 0
        test_loss, test_acc = 0., 0.
        model.eval()
        for X, y in test:
            y_pred = model(X)
            l = loss(y_pred, y)
            test_loss += l.item()
            test_acc += (y_pred.argmax(dim=1) == y).sum().item()
            test_iters += 1
            test_passed += len(X)
        if ep % 5 ==0:   
            print("ep: {}, taked: {:.3f}, train_loss: {}, train_acc: {}, test_loss: {}, test_acc: {}".format(
            ep, time.time() - start, train_loss / train_iters, train_acc / train_passed,
            test_loss / test_iters, test_acc / test_passed)
            )

In [98]:
train_model()

ep: 0, taked: 56.703, train_loss: 0.5324287252911066, train_acc: 0.8110666666666667, test_loss: 0.5665730118751526, test_acc: 0.7918
ep: 5, taked: 56.185, train_loss: 0.2909188391798634, train_acc: 0.8920833333333333, test_loss: 0.44454423040151597, test_acc: 0.8494
ep: 10, taked: 56.625, train_loss: 0.2391800908213955, train_acc: 0.9106833333333333, test_loss: 0.39038146287202835, test_acc: 0.8683
ep: 15, taked: 56.752, train_loss: 0.20400924961698258, train_acc: 0.9233666666666667, test_loss: 0.40426419824361803, test_acc: 0.8737
ep: 20, taked: 56.547, train_loss: 0.17364387400448322, train_acc: 0.93435, test_loss: 0.40046951174736023, test_acc: 0.8802
ep: 25, taked: 56.668, train_loss: 0.14838137225074283, train_acc: 0.9437, test_loss: 0.4776037335395813, test_acc: 0.8698
ep: 30, taked: 56.552, train_loss: 0.1298951634474225, train_acc: 0.95145, test_loss: 0.4482993856072426, test_acc: 0.8788
ep: 35, taked: 56.560, train_loss: 0.11355972147972906, train_acc: 0.9579166666666666, test